<a href="https://colab.research.google.com/github/imp-etus/Homelessness-Study/blob/main/Data%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook prepares Housing & Urban Development (HUD) data for analysis. HUD released a report in 2019 which models a relationship between housing market factors & homelessness. Over the course of this project, we will be attempting to outperform those models. However, in order to do that, the data must be cleaned & major factors normalized.

For the full repository, go [here](https://github.com/imp-etus/Homelessness-Study). 

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

## Load the data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/homelessness/05b_analysis_file_update.csv')

In [ ]:
df_dict = pd.read_csv('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/homelessness/HUD%20TO3%20-%2005b%20Analysis%20File%20-%20Data%20-%20Dictionary.csv')

## Explore the contents of the data set

What variable(s) do we want to predict?

What variables seem useful as predictors?

Are any predictor variables redundant?


### Explore the data sources

Use the dictionary to find the unique `Associated Domain` values.


Make a bar graph of the counts of different data sources described in `Source or Root Variable`. Your graph should have the following features:

1. Order the bars in descending order based on the count.
2. Only include the 10 most common data sources.
3. Orient the plot so that it is easy to read the labels.


In [ ]:
counts = df_dict['Source or Root Variable'].value_counts().head(10)
counts

Census ACS 5-Year Estimates                30
Census Intercensal Population Estimates    28
HUD PIT                                    16
HUD                                         8
HUD HIC                                     8
Eviction Lab at Princeton University        6
See hou_mkt_medrent_acs5yr                  6
See hou_mkt_homeval_acs5yr                  6
See hou_mkt_rentvacancy_acs5yr              5
See dem_soc_singparent_acs5yr               5
Name: Source or Root Variable, dtype: int64

## Convert data types, if necessary

## Select relevant subsets of the data

We will explore a subset of the full data set. Keep at least the following columns. You might include more columns though.

In [ ]:
columns_to_keep = ["year", "cocnumber",
  
                  "pit_tot_hless_pit_hud", "pit_tot_shelt_pit_hud", "pit_tot_unshelt_pit_hud","dem_pop_pop_census",
  
                  "fhfa_hpi_2009", "ln_hou_mkt_medrent_xt", "hou_mkt_utility_xt", "hou_mkt_burden_own_acs5yr_2017", "hou_mkt_burden_sev_rent_acs_2017", "hou_mkt_rentshare_acs5yr_2017", "hou_mkt_rentvacancy_xt", "hou_mkt_density_dummy", "hou_mkt_evict_count", "hou_mkt_ovrcrowd_acs5yr_2017", "major_city", "suburban",
           
                  "econ_labor_unemp_rate_BLS", "econ_labor_incineq_acs5yr_2017", "econ_labor_pov_pop_census_share",
           
                  "hou_pol_hudunit_psh_hud_share", "hou_pol_occhudunit_psh_hud", "hou_mkt_homeage1940_xt",
           
                  "dem_soc_black_census", "dem_soc_hispanic_census", "dem_soc_asian_census", "dem_soc_pacific_census", "dem_pop_child_census", "dem_pop_senior_census", "dem_pop_female_census", "dem_pop_mig_census", "d_dem_pop_mig_census_share", "dem_soc_singadult_xt", "dem_soc_singparent_xt", "dem_soc_vet_xt", "dem_soc_ed_lessbach_xt", "dem_health_cost_dart", "dem_health_excesdrink_chr",
           
                  "env_wea_avgtemp_noaa", "env_wea_avgtemp_summer_noaa", "env_wea_precip_noaa", "env_wea_precip_annual_noaa"]
    

How many variables of each Associated Domain are in the smaller data set?

What are the data sources in the smaller data set?


## Are the data suitable for answering the question?

We want to perform quick exploratory data analysis to determine whether the data are sufficient to answer our question. If the data are not sufficient, we do not want to waste time doing anything that will not be productive.

## Rename columns

In [ ]:
df = df.loc[:, columns_to_keep]

In [ ]:
df = df.rename(columns = {"cocnumber":"coc_number",
                          "pit_tot_hless_pit_hud":"total_homeless", 
                          "pit_tot_shelt_pit_hud":"total_sheltered", 
                          "pit_tot_unshelt_pit_hud":"total_unsheltered",
                          "dem_pop_pop_census":"total_population",
                          "fhfa_hpi_2009":"house_price_index_2009", 
                          "ln_hou_mkt_medrent_xt":"log_median_rent", 
                          "hou_mkt_utility_xt":"utility_costs", 
                          "hou_mkt_burden_own_acs5yr_2017":"percentage_owners_cost_burden_2016", 
                          "hou_mkt_burden_sev_rent_acs_2017":"percentage_renters_severe_cost_burden_2016", 
                          "hou_mkt_rentshare_acs5yr_2017":"share_renters_2016", 
                          "hou_mkt_rentvacancy_xt":"rental_vacancy_rate", 
                          "hou_mkt_density_dummy":"high_housing_density", 
                          "hou_mkt_evict_count":"number_eviction", 
                          "hou_mkt_ovrcrowd_acs5yr_2017":"share_overcrowded_units_2016", 
                          "major_city":"city_or_urban", 
                          "econ_labor_unemp_rate_BLS":"rate_unemployment", 
                          "econ_labor_incineq_acs5yr_2017":"gini_coefficient_2016", 
                          "econ_labor_pov_pop_census_share":"poverty_rate",        
                          "hou_pol_hudunit_psh_hud_share":"share_HUD_units", 
                          "hou_pol_occhudunit_psh_hud":"HUD_unit_occupancy_rate", 
                          "hou_mkt_homeage1940_xt":"share_built_before_1940",           
                          "dem_soc_black_census":"total_black", 
                          "dem_soc_hispanic_census":"total_latino_hispanic", 
                          "dem_soc_asian_census":"total_asian", 
                          "dem_soc_pacific_census":"total_pacific_islander", 
                          "dem_pop_child_census":"total_population_0_19", 
                          "dem_pop_senior_census":"total_population_65_plus", 
                          "dem_pop_female_census":"total_female_population", 
                          "dem_pop_mig_census":"net_migration", 
                          "d_dem_pop_mig_census_share":"migration_4_year_change", 
                          "dem_soc_singadult_xt":"proportion_one_person_households", 
                          "dem_soc_singparent_xt":"share_under_18_with_single_parent", 
                          "dem_soc_vet_xt":"share_veteran_status", 
                          "dem_soc_ed_lessbach_xt":"share_no_bachelors", 
                          "dem_health_cost_dart":"medicare_reimbursements_per_enrollee", 
                          "dem_health_excesdrink_chr":"percentage_excessive_drinking",
                          "env_wea_avgtemp_noaa":"average_Jan_temperature", 
                          "env_wea_avgtemp_summer_noaa":"average_summer_temperature", 
                          "env_wea_precip_noaa":"total_Jan_precipitation", 
                          "env_wea_precip_annual_noaa":"total_annual_precipitation"})

## Quality Control

Check for out-of-range values or values that do not match what we want to analyze. Either set values to `NaN` or remove the observations, as appropriate.

In [ ]:
df.agg(['min', 'max']).round(2).transpose()

,min,max
year,2010,2017
coc_number,AK-500,WY-500
total_homeless,7.0,76501.0
total_sheltered,3.0,72565.0
total_unsheltered,0.0,42828.0
total_population,29344,11058958
house_price_index_2009,-26.5,63.86
log_median_rent,1.24,2.87
utility_costs,5.6,22.6
percentage_owners_cost_burden_2016,8.07,22.96


## Identify and deal with missing values

We then filter to only taking data from 2017 & only where total homeless has a valid input, because that's what we're trying to predict.

In [ ]:
df_updated = df.dropna(subset=['total_homeless']).loc[df['year'] == 2017]

In [ ]:
# convert homeless rates to per 10k, convert other things to %s
df_per10ks = {'total_homeless', 'total_sheltered', 'total_unsheltered'}
df_rates = {'total_homeless', 'total_sheltered', 'total_unsheltered', "total_black", "total_latino_hispanic", "total_asian", "total_pacific_islander", "total_population_0_19", "total_population_65_plus", "total_female_population"}

In [ ]:
for col in df_rates:
  df_updated[col] = df_updated[col] / df_updated['total_population']

In [ ]:
for col in df_per10ks:
  df_updated[col] = df_updated[col] * 10_000

In [ ]:
df_updated = df_updated.rename(columns={'total_homeless':'homeless_per_10k', 
'total_sheltered':'sheltered_per_10k', 
'total_unsheltered':'unsheltered_per_10k',
'total_black':'rate_black', 
'total_latino_hispanic':'rate_latino_hispanic', 
'total_asian':'rate_asian', 
'total_pacific_islander':'rate_pacific_islander', 
'total_population_0_19':'rate_0_19', 
'total_population_65_plus':'rate_65_plus', 
'total_female_population':'rate_female'})

## Create relevant derived variables as new columns

We may already know that we want to process the data to create new variables from the existing variables. However, we often start analyzing the data and realize that it is useful to create new variables derived from the existing variables. Or, we might not create any new columns. It is fine to return to modify this step after exploring the data further.

## Export the clean .csv file

Export the clean data set

In [ ]:
from google.colab import files
df_updated.to_csv('homelessness_data.csv', encoding='utf-8-sig', index=False)
files.download('homelessness_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>